<a href="https://colab.research.google.com/github/repairedserver/Test/blob/master/%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
cd/content/gdrive/My Drive/deeplearningbro/deeplearningbro/pytorch

/content/gdrive/My Drive/deeplearningbro/deeplearningbro/pytorch


In [11]:
import pandas as pd #데이터프레임
import numpy as np
from sklearn.model_selection import train_test_split

#ANN
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

#Cross Validation
from sklearn.model_selection import KFold

#Loss
from sklearn.metrics import mean_squared_error

#Plot
import matplotlib.pyplot as plt

In [13]:
df = pd.read_csv('./data/reg.csv', index_col=[0])

In [21]:
X = df.drop('Price', axis=1).to_numpy() #타겟값 제외 넘파이 배열
Y = df['Price'].to_numpy().reshape((-1, 1))

In [15]:
class TensorData(Dataset):
  def __init__(self, x_data, y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.FloatTensor(y_data)
    self.len = self.y_data.shape[0]

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.7)
trainset = TensorData(X_train, Y_train)
testset = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

In [20]:
class Regressor(nn.Module):
  def __init__(self):
    super().__init__() #모델 연산 정의
    self.fc1 = nn.Linear(13, 50, bias=True)
    self.fc2 = nn.Linear(50, 30, bias=True)
    self.fc3 = nn.Linear(30, 1, bias=True)

  def forward(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)

    return x

In [18]:
KFold = KFold(n_splits=3, shuffle=True)

In [19]:
criterion = nn.MSELoss()

In [23]:
def evaluation(dataloader):
  predictions = torch.tensor([], dtype=torch.float)
  actual = torch.tensor([], dtype=torch.float)

  with torch.no_grad():
    model.eval() #평가시 .eval() 반드시 사용
    for data in dataloader:
      inputs, values = data
      outputs = model(inputs)

      predictions = torch.cat((predictions, outputs), 0)
      actual = torch.cat((actual, values), 0)

    predictions = predictions.numpy()
    actual = actual.numpy()
    rmse = np.sqrt(mean_squared_error(predictions, actual))
    model.train()
    return rmse

In [26]:
validation_loss = []
for fold, (train_idx, val_idx) in enumerate(KFold.split(trainset)):
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
  val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

  #sampler를 이용한 DataLoader 정의
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, sampler=train_subsampler)
  valloader = torch.utils.data.DataLoader(trainset, batch_size=32, sampler=val_subsampler)

  model = Regressor()
  optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-7)

  for epoch in range(400):
    for data in trainloader: #무작위로 섞인 32개 데이터가 있는 배치가 하나씩
      inputs, values = data
      optimizer.zero_grad() #최적화 초기화

      outputs = model(inputs) #모델에 입력값 대입후 예측값
      loss = criterion(outputs, values) #손실
      loss.backward() #손실함수 기준으로 역전파
      optimizer.step() #역전파 진행후 가중치

    train_rmse = evaluation(trainloader) #학습 데이터의 RMSE
    val_rmse = evaluation(valloader)
    print("k-fold", fold,"Train Loss = %.4f, Validation Loss = %.4f" %(train_rmse, val_rmse))
    validation_loss.append(val_rmse)

validation_loss = np.array(validation_loss)
mean = np.mean(validation_loss)
std = np.std(validation_loss)
print("Validation Score: %.4f, +-%.4f" %(mean, std))

k-fold 0 Train Loss = 0.2893, Validation Loss = 0.2783
k-fold 0 Train Loss = 0.2370, Validation Loss = 0.2603
k-fold 0 Train Loss = 0.1763, Validation Loss = 0.1723
k-fold 0 Train Loss = 0.1484, Validation Loss = 0.1528
k-fold 0 Train Loss = 0.1707, Validation Loss = 0.1824
k-fold 0 Train Loss = 0.1323, Validation Loss = 0.1418
k-fold 0 Train Loss = 0.1372, Validation Loss = 0.1439
k-fold 0 Train Loss = 0.1282, Validation Loss = 0.1415
k-fold 0 Train Loss = 0.1291, Validation Loss = 0.1393
k-fold 0 Train Loss = 0.1274, Validation Loss = 0.1358
k-fold 0 Train Loss = 0.1175, Validation Loss = 0.1289
k-fold 0 Train Loss = 0.1261, Validation Loss = 0.1374
k-fold 0 Train Loss = 0.1179, Validation Loss = 0.1252
k-fold 0 Train Loss = 0.1147, Validation Loss = 0.1222
k-fold 0 Train Loss = 0.1127, Validation Loss = 0.1201
k-fold 0 Train Loss = 0.1089, Validation Loss = 0.1172
k-fold 0 Train Loss = 0.1234, Validation Loss = 0.1330
k-fold 0 Train Loss = 0.1165, Validation Loss = 0.1273
k-fold 0 T

In [28]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=False)
train_rmse = evaluation(trainloader)
test_rmse = evaluation(testloader)

print("Train RMSE: %.4f" %train_rmse)
print("Test RMSE: %.4f" %test_rmse)

Train RMSE: 0.1189
Test RMSE: 0.1389
